In [21]:
# standard imports
import datetime
import os
import sys


from subprocess import check_output

# third party imports
import matplotlib.pyplot as plt # MATLAB like plotting framework
import numpy as np # linear algebra
import pandas as pd # Data ETL, csv file I/O

# src directory imports
module_path = os.path.abspath(os.path.join('src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.helper_functions import load_dataset

%matplotlib inline

#### Check files in data folder

In [51]:
print(check_output(["ls", "./data"]).decode("utf-8"))

Kickstarter001.csv
Kickstarter002.csv
Kickstarter003.csv
Kickstarter004.csv
Kickstarter005.csv
Kickstarter006.csv
Kickstarter007.csv
Kickstarter008.csv
Kickstarter009.csv
Kickstarter010.csv
Kickstarter011.csv
Kickstarter012.csv
Kickstarter013.csv
Kickstarter014.csv
Kickstarter015.csv
Kickstarter016.csv
Kickstarter017.csv
Kickstarter018.csv
Kickstarter019.csv
Kickstarter020.csv
Kickstarter021.csv
Kickstarter022.csv
Kickstarter023.csv
Kickstarter024.csv
Kickstarter025.csv
Kickstarter026.csv
Kickstarter027.csv
Kickstarter028.csv
Kickstarter029.csv
Kickstarter030.csv
Kickstarter031.csv
Kickstarter032.csv
Kickstarter033.csv
Kickstarter034.csv
Kickstarter035.csv
Kickstarter036.csv
Kickstarter037.csv
Kickstarter038.csv
Kickstarter039.csv
Kickstarter040.csv
Kickstarter041.csv
Kickstarter042.csv



#### Load dataframe

In [47]:
df = load_dataset()

In [48]:
df.head()

,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,...,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,2081133073,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Dracula: The Kisses... Help Scary Little Girls...,Dracula: The Kisses… is an all-female producti...,8000.0,2507.0,failed,dracula-the-kisses-help-scary-little-girls-fly...,False,GB,...,"{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",False,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,262077000,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Historial Women,The Power of Women and Their Contributions to ...,2000.0,19.0,failed,historial-women,False,US,...,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",False,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,1541121253,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",The Sequel: a comedy with puppets,Our play The Sequel has a 2 week run at The Ca...,500.0,1.0,failed,the-sequel-a-comedy-with-puppets,False,GB,...,"{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",False,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
3,489010822,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Young blind actors have a life-changing opport...,Four blind youth from Los Angeles based Theatr...,15000.0,2095.0,failed,young-blind-actors-have-a-life-changing-opport...,False,US,...,"{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",False,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
4,702028718,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Drama School Fund,Raising money to send a young Actress to Drama...,10600.0,11.0,failed,drama-school-fund,False,GB,...,"{""country"":""GB"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...",False,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172833 entries, 0 to 172832
Data columns (total 32 columns):
id                        172833 non-null int64
photo                     172833 non-null object
name                      172832 non-null object
blurb                     172821 non-null object
goal                      172833 non-null float64
pledged                   172833 non-null float64
state                     172833 non-null object
slug                      172833 non-null object
disable_communication     172833 non-null bool
country                   172833 non-null object
currency                  172833 non-null object
currency_symbol           172833 non-null object
currency_trailing_code    172833 non-null bool
deadline                  172833 non-null int64
state_changed_at          172833 non-null int64
created_at                172833 non-null int64
launched_at               172833 non-null int64
staff_pick                172833 non-null bool
backers_count   

### Convert from unix time to pandas datetime

In [50]:
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['state_changed_at'] = pd.to_datetime(df['state_changed_at'],unit='s')
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')